# NP-Hard Problems

The purpose of this assignment is to familiarize yourself with different approaches to solving NP-hard problems in practice, especially via integer programming.

As in previous assignments, we will use [NetworkX](https://networkx.github.io/) to manipulate graphs and [PuLP](http://pythonhosted.org/PuLP/) to solve linear and integer programs.

In [142]:
import networkx as nx
import pulp

We will use two graph datasets for this assignment, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [143]:
karate = nx.read_gml('karate.gml')
power = nx.read_gml('power.gml')

#
# The nx.read_gml() function was giving me an error, which I believe was
# due to the version of NetworkX I am using.  I wrote this function to 
# read in the gml files in order to avoid having to revert my NetworkX 
# installation to a previous version (others appear to have had the same
# problem - see Piazza @243)
#
# def my_read_gml(gml_file):
    
#     G = nx.Graph()
    
#     file = open(gml_file)
#     temp_src = -1
#     for line in file:
#         tokens = line.strip().split()
#         if tokens[0] == 'id':
#             G.add_node(int(tokens[1]))
        
#         if tokens[0] == 'source':
#             if(temp_src) != -1:
#                 print "ERROR: Unexpected formatting in file"                
#             temp_src = int(tokens[1])
        
#         if tokens[0] == 'target':
#             G.add_edge(temp_src, int(tokens[1]))
#             temp_src = -1

#     return G

# karate = my_read_gml('karate.gml')
# power = my_read_gml('power.gml')



# Test graph 1

# G = nx.Graph()

# for i in range(0, 24):
#     G.add_node(i)
    
# for i in range(0, 12):
#     G.add_edge(i, (i + 1)%12)
#     G.add_edge(i, i + 12)

# for i in range(12, 16):
#     G.add_edge(i, i + 4)
#     G.add_edge(i + 4, i + 8)
#     G.add_edge(i + 8, i)

# Max size indepdenent set should be 9


# Test graph 2

# G = nx.Graph()

# for i in range(0, 12):
#     G.add_node(i)

# for i in range(0, 7):
#     G.add_edge(i, (i + 1)%7)

# G.add_edge(0, 7)
# G.add_edge(1, 8)
# G.add_edge(2, 8)
# G.add_edge(3, 9)
# G.add_edge(4, 9)
# G.add_edge(5, 10)
# G.add_edge(6, 10)
# G.add_edge(7, 8)
# G.add_edge(7, 11)
# G.add_edge(9, 11)
# G.add_edge(10, 11)

# Max size indepdenent set should be 5

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 


## Independent Set

Recall that an *independent set* in a graph is a set of nodes such that no two nodes are adjacent (connected by an edge). Finding the maximum size of an independent set in a graph is an NP-hard problem, so there is no known polynomial-time algorithm to solve it.

### Problem 1

Provide an *integer program* formulation for this problem. This should look identical to a linear program formulation, except you will restrict the variables corresponding to the nodes to be integer instead of fractional.

The graph is denoted $G = (V, E)$. For this problem, you should use a binary variable $x_i \in \{0,1\}$ for each node $i \in V$, and you shouldn't need any other variables.

#### Solution:

Integer program with the following input variables, linear objective function and linear constraints:

Input Variables:

$x_i$: a binary variable that equals 1 if node $i$ is in our independent set, and 0 otherwise

Linear Objective Function:

$ \max \sum_{i \in V} x_i$

Subject to the linear constraints:

$x_i \in \{0,1\}$  $\forall i \in V $ 

$x_i + x_j \leq 1$  $\forall (i,j) \in E$



### Problem 2

Implement a function that solves the integer program given a graph as input.

In [144]:
def independent_set_ip(graph):
    """Computes a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    
    nodes = []
    edges = []
    
    # extract nodes from graph
    for n in graph.nodes():
        nodes.append(n)        
        
    # extract edges from graph
    for (u, v) in graph.edges():
        edges.append((u, v))
    
    # indicator variables: x_i = 1 if node i is in MSIS, x = 0 otherwise
    x = pulp.LpVariable.dicts("Indicators", nodes, None, None, pulp.LpInteger)
    
    # problem statement     
    prob = pulp.LpProblem("Maximum Size Independent Set", pulp.LpMaximize)
 
    # objective function
    prob += pulp.lpSum([x[n] for n in nodes]) # sum of x's 
    
    # integer constraints
    for n in nodes:
        x[n].bounds(0, 1) # either 0 or 1 since x's are integers
    
    # independence constraints                                         
    for (u,v) in edges:
        prob += (x[u] + x[v] <= 1)
    
    # solve the linear program     
    prob.solve()
     
    solution = []

    for n in nodes:    
        solution.append((n, int(pulp.value(x[n]))))        
    
    # return value of objective function as integer
    return solution    


The following code outputs the size of the sets computed by your function.

In [145]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [146]:
karate_ind_set = independent_set_ip(karate)
print "Size of karate set = ", set_weight(karate_ind_set)
power_ind_set = independent_set_ip(power)
print "Size of power set = ", set_weight(power_ind_set)

Size of karate set =  20
Size of power set =  2738


### Problem 3

Take the *linear programming relaxation* of your integer program and implement a function to solve it. This simply means that in your integer program, you should replace each constraint $x_i \in \{0,1\}$ with $0 \leq x_i \leq 1$.

In [147]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    
    nodes = []
    edges = []
    
    # extract nodes from graph
    for n in graph.nodes():
        nodes.append(n)        
        
    # extract edges from graph
    for (u, v) in graph.edges():
        edges.append((u, v))
    
    # indicator variables: x_i = 1 if node i is in MSIS, x = 0 otherwise
    x = pulp.LpVariable.dicts("Indicators", nodes, None, None, pulp.LpContinuous)
    
    # problem statement     
    prob = pulp.LpProblem("Maximum Size Independent Set", pulp.LpMaximize)
 
    # objective function
    prob += pulp.lpSum([x[n] for n in nodes]) # sum of x's 
    
    # integer constraints
    for n in nodes:
        x[n].bounds(0, 1) # between 0 and 1 since x's are continuous
    
    # independence constraints                                                
    for (u,v) in edges:
        prob += (x[u] + x[v] <= 1)
    
    # solve the linear program     
    prob.solve()
     
    solution = []

    for n in nodes:    
        solution.append((n, pulp.value(x[n])))        
    
    # return value of objective function as float
    return solution    
    

Let's see how the LP solutions compare to those of the IP.

In [148]:
karate_ind_set_relax = independent_set_lp(karate)
print "Value of karate set = ", set_weight(karate_ind_set_relax)
power_ind_set_relax = independent_set_lp(power)
print "Value of power set = ", set_weight(power_ind_set_relax)

Value of karate set =  20.5
Value of power set =  2758.0


A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $a$ nodes which have $b$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $a - b$.

Implement this rounding procedure.

In [149]:
def round_solution(solution, graph):
    """Finds the subgraph corresponding to the rounding of
    a solution to the independent set LP relaxation.
    
    Args:
      - solution (list[(int, float)]): LP solution
      - graph (nx.Graph): the original graph
      
    Returns:
        (nx.Graph) The subgraph corresponding to rounded solution
    
    """
    # TODO: implement function   
    node_set = []
    
    for i,x_i in solution:
        if x_i > .5:
            node_set.append(i)            

    for n in graph.nodes():
        if n not in node_set:
            graph.remove_node(n)
            
    return graph

The following function assesses the quality of the heuristic approach.

In [150]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set
    
    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [151]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print "Quality of karate rounded solution = {:.0f}%".format(karate_quality*100)

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print "Quality of power rounded solution = {:.0f}%".format(power_quality*100)

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
